In [2]:
import geolib

girder_url = 'http://anka:8080'
api_key = '3x8bBbwik9TLlS9lJU8Z8qNeNLfQQsm9QPU7FrkS'
girder = geolib.connect(girder_url=girder_url, apikey=api_key)


In [4]:
dataset_path = 'Public/PH_community_distribution_map.tif'
#dataset_path = 'Public/PH_plant_height_map.tif'
data_url = girder.lookup_url(dataset_path)
dataset = geolib.create(data_url)
dataset._epsg = '4326'
dataset._setdatatype(geolib.types.RASTER)
dataset.opacity = 0.8

# Fix metadata
metadata = dataset.get_metadata()
dataset._metadata['bounds']['coordinates'] = [[
    [-106.95242631255823, 38.92484206381101],
    [-106.95224175189571, 38.91810990322611],
    [-106.94362420673949, 38.91825374356009],
    [-106.94380795397146, 38.92498593850482]
]]

scene1 = geolib.show(dataset)
display(scene1)

scene(center={'y': 0.0, 'x': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [5]:
import ipywidgets as widgets
z1 = widgets.FloatSlider(min=1, max=18, value=scene1.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((z1, 'value'), (scene1, 'zoom'))
display(z1)


FloatSlider(value=15.5, layout=Layout(width='95%'), max=18.0, min=1.0)

In [6]:
import geojson

# Create vector object for dataset bounds
meta = dataset.get_metadata()
bounds = meta.get('bounds',{})
# assert bounds, 'Dataset bounds missing'

bounds_feature = geojson.Feature(geometry=bounds, properties={'fillColor': 'black', 'fillOpacity': 0.1})
bounds_object = geolib.create(bounds_feature)

# Compute simple crop region inside the dataset
coords = bounds.get('coordinates')[0]
xvals, yvals = zip(*coords)
xmin = min(xvals); ymin = min(yvals)
xmax = max(xvals); ymax = max(yvals)

# Compute center coordinates
x = (xmin + xmax) / 2.0
y = (ymin + ymax) / 2.0

# Use percentage of height & width
dx = 0.12 * (xmax - xmin)
dy = 0.18 * (ymax - ymin)
poly = [[x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]]

crop_geom = geojson.Polygon([poly])
crop_feature = geojson.Feature(geometry=crop_geom, properties={'fillColor': 'magenta', 'fillOpacity': 0.6})
crop_object = geolib.create(crop_feature)
scene2 = geolib.show([crop_object, bounds_object])
display(scene2)

scene(center={'y': 0.0, 'x': 0.0}, layout=Layout(align_self='stretch', height='400px'))

In [7]:
import ipywidgets as widgets
z2 = widgets.FloatSlider(min=1, max=18, value=scene2.zoom, layout=dict(width='95%'))
newlink = widgets.jslink((z2, 'value'), (scene2, 'zoom'))
display(z2)

FloatSlider(value=15.5, layout=Layout(width='95%'), max=18.0, min=1.0)